In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Custom Libs
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('../')))
from ridgepy.kalman_filter_network import KalmanFilterNetwork
from ridgepy.kalman_filter_network import KalmanFilterMode

# Input
This is a simple example with a single mode input

In [ ]:
frequency_sample = 16000.0
start_time       = 0.0
end_time         = 0.2
time             = np.arange(start_time, end_time, 1./frequency_sample)

F           = 300.0
A           = 1.0
B           = 0.5
phase       = np.mod(2*np.pi*F*time, 2*np.pi)
observation = A*np.cos(phase) + B*np.sin(phase)

data_df = pd.DataFrame({
    'time': time,
    'phase': phase,
    'observation': observation
})

data_df.head()

# Kalman Filter Network
The network is a collection of modes.

Each mode has a frequency, a set of coefficients, and a set of covariance
matrices.

The number of frequencies and their values are the key parameters in the
network. In a Discrete Fourier Transform, the frequencies are determined from a
linear spacing on teh region [0, Nf/2]. Other strategies include Mel frequency
spacing, or whatever else.

In [ ]:
# Simple example with frequencies at 50, 100, ..., 500 Hz.
frequencies = np.arange(50, 10*50, 50)
modes = []
for frequency in frequencies:
    # The signal noise covariance is a 2x2 matrix.
    signal_noise_covariance = np.zeros((2,2))
    signal_noise_covariance[0][0] = 0.0001
    signal_noise_covariance[1][1] = 0.0001

    signal_error_covariance = np.zeros((2,2))
    signal_error_covariance[0][0] = 1.0
    signal_error_covariance[1][1] = 1.0

    # The observation noise covariance is a scalar.
    observation_noise_covariance = 10.0

    # The coefficients are a 2 X 1 matrix.
    sin_coefficient = 0.0
    cos_coefficient = 0.0

    mode = KalmanFilterMode(
        frequency,
        sin_coefficient,
        cos_coefficient,
        signal_error_covariance,
        signal_noise_covariance,
        observation_noise_covariance
    )
    modes.append(mode)

In [ ]:
frequency_sample = 16000.
kf_network = KalmanFilterNetwork(modes)

# Network Updates
In the code below, the network update is called recursively on the next
observation.  In this example, the sampling frequency of the observations is
constant, but that does not need to be true.

In [ ]:
time_step_data = []

for ndx in range(0, len(data_df)):

    ## Update the prior
    kf_network.prior_update(frequency_sample)

    ## Update the posterior
    kf_network.posterior_update(data_df.at[ndx, 'observation'])

    ## Change the logging below to use a dictionary
    current_parameters = kf_network.current_parameters()

    time_step_data.append(current_parameters)

results_df = pd.DataFrame(time_step_data)

In [ ]:
results_df.head()

# Results
The typical solution would be to use a spectrogram. That is shown in the first
image below, along with the results from the Kalman Filter Network

In [ ]:
fs = 1./(data_df.at[1, 'time'] - data_df.at[0, 'time'])
plt.specgram(data_df['observation'], NFFT=256, Fs=fs, cmap='viridis')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram')
plt.colorbar(label='Power spectral density (dB/Hz)')
plt.show()

In [ ]:

magnitudes = np.array(results_df['magnitudes'].apply(np.array).tolist())
magnitudes_db = np.log10(magnitudes) * 20.0

frequencies = np.array(results_df['frequencies'].apply(np.array).tolist())
times = data_df['time']
print(times.shape, frequencies.shape, magnitudes_db.shape)
plt.pcolormesh(times, frequencies.T, magnitudes_db.T, cmap='viridis')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram')
plt.colorbar(label='Power spectral density (dB/Hz)')
plt.show()